## Search Engine With Tools and Agents

In [2]:
## Arxiv  --Research
## Toosl Creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import  WikipediaAPIWrapper, ArxivAPIWrapper



In [3]:
# use inbuilt tool for wikipedia search

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)  
wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [5]:
tools = [wiki, arxiv]

In [6]:
## custom tool Rag
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documnets = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(documnets, OpenAIEmbeddings())
retriver = vectorstore.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001927E8EC280>, search_kwargs={})

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver,"langsmith-search","Search any information about langsmith search")
retriver_tool.name

'langsmith-search'

In [11]:
tools = [wiki, arxiv, retriver_tool]

In [12]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\LangChain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about langsmith search', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001927EA66440>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001927E8EC280>, search_kwar

In [13]:
### Run all tools with agents and LLM models

### Tools,LLM--> AgentExecutor
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
import openai
import os

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")


llm = ChatGroq(groq_api_key=groq_api_key,model = "llama3-70b-8192")

In [14]:
### Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
##Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [16]:
##  AgentExecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [17]:
agent_executor.invoke({"input":"What is LangSmith? "})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'what is langsmith'}`


Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic â€” score application performance and get human feedback on your data.Prompt Eng

{'input': 'What is LangSmith? ',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It provides features such as observability, evaluation, and prompt engineering to help developers build, monitor, and improve their LLM applications.'}

In [20]:
agent_executor.invoke({"input":"What is MachineLearning in detail in points please? "})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witHere is a detailed explanation of Machine Learning in points:

* Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms.
* These algorithms can learn from data and generalize to unseen data, and thus perform tasks without being explicitly programmed.
* Machine learning is a type of artificial intelligence that enables computers to learn from experience and improve their performance on a task without being explicitly programmed.
* It involves training algorithms on data and using that data to make predictions or decisions.
* Machine learning is a key technology 

{'input': 'What is MachineLearning in detail in points please? ',
 'output': 'Here is a detailed explanation of Machine Learning in points:\n\n* Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms.\n* These algorithms can learn from data and generalize to unseen data, and thus perform tasks without being explicitly programmed.\n* Machine learning is a type of artificial intelligence that enables computers to learn from experience and improve their performance on a task without being explicitly programmed.\n* It involves training algorithms on data and using that data to make predictions or decisions.\n* Machine learning is a key technology driving many AI applications, including image and speech recognition, natural language processing, and expert systems.\n* There are several types of machine learning, including:\n\t+ Supervised learning: The algorithm is trained on labeled data, where the correct outp